# Исследование: музыка больших городов

## Введение

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс Музыки вы сравним поведение пользователей двух столиц.

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей получим из файла `yandex_df.csv`. О качестве данных ничего не известно. 

## Этап 1. Обзор данных и их предобработка <a id='step_1'></a>

### Импорт библиотек <a id='step_1.1'></a>

In [1]:
import pandas as pd
import numpy as np
from IPython import display

import matplotlib.pyplot as plt

### Общая информация о данных <a id='step_1.2'></a>

Изучим данные, предоставленные сервисом для проекта.

In [2]:
# Прочитаем файл yandex_music_project.csv с информацией о логах в приложении и сохраним его в переменной df
df = pd.read_csv('/datasets/yandex_music_project.csv')
df.head()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [3]:
# Получим общую информацию о данных таблицы df
display.display(data=df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Рассмотрим полученную информацию подробнее. Всего в таблице 7 столбцов и 65 079 строк, тип данных у каждого столбца - < object >.

Подробно разберём, какие в таблице столбцы, и какую информацию они содержат:
* `userID` — идентификатор пользователя;
* `Track` — название трека;
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город, в котором происходило прослушивание;
* `time` — время, в которое пользователь слушал трек;
* `Day` — день недели.

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения. Это может быть связано с техническими проблемами выгрузки данных, либо некорректно внесённых данных в саму базу данных. Также видим, что для удобства анализа данных столбцы требуют приведения в необходимый вид (удаление пробелов, строковые значения). Изменения типов данных не требуется.

### Обработка пропусков и дубликатов, проверка данных <a id='step_1.3'></a>

#### Замена названий стобцов

При анализе информации об исходных датасетах мы обратили внимание на то, что для дальнейшего анализа необходимо привести названия столбцов таблицы df к удобному для анализа виду.

In [4]:
# Получим перечень названий столбцов
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

В названиях столбцов есть пробелы, которые могут затруднять доступ к данным. Кроме того, названия набраны в разных регистрах и не полностью отражают в краткой форме, какая информация в них содержится, что также может усложнить работу с данными. Переименуем столбцы и проверим результат.

In [5]:
df.set_axis(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday'],
                       axis='columns', inplace=True)
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

Все названия столбцов имеют подходящий для анализа вид.

#### Обработка пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [6]:
df.isnull().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Пустые значения свидетельствуют о том, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем, не назван конкретный исполнитель народной песни. Хуже, если возникли проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.

Заменяем пропущенные значения в столбцах с названием трека и исполнителя на строку *'unknown'*.

In [7]:
df['track_name'] = df['track_name'].fillna('unknown')
df['artist_name'] = df['artist_name'].fillna('unknown')

В столбце с жанрами есть пустые значения, но их не очень много. Рассчитаем процент пропусков по столбцу `genre_name` к общей выборке.

In [8]:
df['genre_name'].isna().mean()

0.018408395949538255

Пропуски составляют всего 2% от общей выборки. Мы можем их удалить, после чего проверить, что пропусков в данных не осталось.

In [9]:
df.dropna(subset = ['genre_name'], inplace = True)
df.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

#### Обработка дубликатов

Необходимо установить наличие дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.

In [10]:
df.duplicated().sum()

3755

In [11]:
df = df.drop_duplicates().reset_index(drop=True)

In [12]:
df.duplicated().sum()

0

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

Также отметим, что помимо "явных" дубликатов, могут встречаться "неявные". Чтобы убедиться в их отсутствии, следует проверить значения в столбцах.

#### Проверка данных

Изучим значения, которые встречаются в каждом их столбцов таблицы `df`.

##### Идентификатор пользователя <a id='id'></a>

In [13]:
df['user_id'].describe()

count        60126
unique       41315
top       7D166C63
freq            51
Name: user_id, dtype: object

В данных встречается 60 126 записей, в которых 41 315 уникальных пользователей.

##### Название трека <a id='track'></a>

In [14]:
df['track_name'].describe()

count       60126
unique      46910
top       unknown
freq          536
Name: track_name, dtype: object

In [15]:
df['track_name'].unique()[:20]

array(['Kamigata To Boots', 'Delayed Because of Accident',
       'Funiculì funiculà', 'Dragons in the Sunset', 'Soul People',
       'Преданная', 'True', 'Feeling This Way',
       'И вновь продолжается бой', 'Pessimist', 'Gool la Mita',
       'Is There Anybody Out There? (Panoramic Paralysis) Mix',
       'Мина син генэ кирэк', 'After School Special',
       'Make Love Whenever You Can', 'SLAVES OF FEAR', 'Веснянка',
       'Pat-a-Cake', 'Sweetback', "Ya'll In Trouble"], dtype=object)

В данных встречается 46 910 уникальных композиций по названию трека.

In [16]:
(df
     .groupby('user_id')['track_name']
     .nunique()
     .reset_index()
     .query('track_name > 10')
     .sort_values(by='track_name', ascending=False)
     .reset_index(drop=True)
)

,user_id,track_name
0,E8339398,42
1,B5496034,34
2,D6A03F1E,32
3,B851E1A5,32
4,53B9A51D,28
...,...,...
99,AA950A3A,11
100,9A9E39A0,11
101,71255BAD,11
102,60864740,11


Отметим, что 104 пользователя слушали более 10 треков с разными названиями. Некоторые из них слушали до 42, 34, 32 треков с разными названиями.

In [17]:
(len(df
         .groupby('user_id')['track_name']
         .nunique()
         .reset_index()
         .query('track_name == 1'))
)

31762

Также у нас имеется 31 762 пользователей, которые включали либо только один трек, либо треки, имеющие одинаковые названия.

##### Имя исполнителя <a id='artist'></a>

In [18]:
df['artist_name'].describe()

count       60126
unique      43344
top       unknown
freq         5990
Name: artist_name, dtype: object

In [19]:
df['artist_name'].unique()[:20]

array(['The Mass Missile', 'Andreas Rönnberg', 'Mario Lanza',
       'Fire + Ice', 'Space Echo', 'IMPERVTOR', 'Roman Messer',
       'Polina Griffith', 'unknown', 'Shireen Abdul Wahab',
       'Pink Floyd Floydhead', 'Ильдар Хакимов', 'Detroit Grand Pubahs',
       'Arabesque', 'HEALTH', 'Веданъ Колодъ', 'Mother Goose Club',
       'The Fabulous Three',
       'Lil Tee Chill Tank Young Buck Brother Mohammed and Tara',
       'Exotype'], dtype=object)

В данных встречается 43 344 уникальных исполнителя.

In [20]:
(df
     .groupby('user_id')['artist_name']
     .nunique()
     .reset_index()
     .query('artist_name > 10')
     .sort_values(by='artist_name', ascending=False)
     .reset_index(drop=True)
)

,user_id,artist_name
0,E8339398,38
1,B5496034,34
2,B851E1A5,32
3,D6A03F1E,31
4,3E4BFAA7,30
...,...,...
93,E4F8C670,11
94,794B409F,11
95,D540AF61,11
96,D2CE494E,11


Отметим, что 98 пользователей слушали более 10 треков с разными исполнителями. Некоторые из них слушали до 38, 34, 32 треков с разными исполнителями.

In [21]:
(len(df
         .groupby('user_id')['artist_name']
         .nunique()
         .reset_index()
         .query('artist_name == 1'))
)

31885

Также у нас имеется 31 885 пользователей, которые включали либо только один трек, либо треки, принадлежащие одному исполнителю.

##### Название жанра <a id='genre'></a>

In [22]:
df['genre_name'].describe()

count     60126
unique      289
top         pop
freq       8323
Name: genre_name, dtype: object

In [23]:
df['genre_name'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Сохраним список уникальных значений столбца с жанрами в переменной `genres_list`. Отметим, что некоторые жанры выглядят как неявные дубликаты. Их стоит обработать.

In [24]:
genres_list = df['genre_name'].unique()

Объявим функцию `find_genre()` для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.

In [25]:
def find_genre(genre):
    count_genre = 0
    for row in genres_list:
        if row == genre:
            count_genre += 1
    return count_genre

Вызовем функцию `find_genre()` для поиска различных вариантов названия жанра хип-хоп в таблице. Поищем варианты:
- hiphop
- hip
- hop
- hip-hop

In [26]:
find_genre('hiphop')

1

In [27]:
find_genre('hip')

1

In [28]:
find_genre('hop')

1

In [29]:
find_genre('hip-hop')

1

Объявим функцию `find_right_genre()`, которая заменяет неправильное название нужного жанра в столбце `genre_name` на правильное и проверяет успешность выполнения замены.

Так исправляем все варианты написания, которые выявила проверка.

In [30]:
def find_right_genre(df, right, wrong):
    df['genre_name'] = df['genre_name'].replace(wrong, right)
    result = df[df['genre_name'] == wrong]['genre_name'].count()
    return result

In [31]:
find_right_genre(df, 'hiphop', 'hip')

0

Проведём аналогичные процедуры для жанров 'africa' (другое название 'afrikaans'), 'arabesk' (другое название 'arabic'), 'latin' (другое название 'latino'), 'poprussian' (другое название 'ruspop'), 'soul' (другое название 'soulful'), 'türk' (другое название 'türkçe'), 'electronic' (другое название 'электроника').

In [32]:
find_right_genre(df, 'afrikaans', 'africa')

0

In [33]:
find_right_genre(df, 'latino', 'latin')

0

In [34]:
find_right_genre(df, 'ruspop', 'poprussian')

0

In [35]:
find_right_genre(df, 'soul', 'soulful')

0

In [36]:
find_right_genre(df,  'türkçe', 'türk')

0

In [37]:
find_right_genre(df,  'electronic', 'электроника')

0

Проверим, сколько уникальных значений жанров у нас осталось.

In [38]:
df['genre_name'].describe()

count     60126
unique      282
top         pop
freq       8323
Name: genre_name, dtype: object

Таким образом, в данных имеются композиции, принадлежащие к 280 разным жанрам.

##### Город <a id='city'></a>

In [39]:
df['city'].unique()

array(['Saint-Petersburg', 'Moscow'], dtype=object)

In [40]:
(df
     .groupby('city')['user_id']
     .nunique()
     .reset_index()
)

,city,user_id
0,Moscow,29043
1,Saint-Petersburg,12305


Прослушивающих музыку из Москвы в выборке 29 043 пользователя, а из Санкт-Петербурга - 12 305 человек. Поскольку число уникальных пользователей в выборке равно 41 315, а суммарное значение пользователей из Москвы и Санкт-Петербурга выше, некоторые пользователи слушали музыку и в Москве, и в Санкт-Петербурге. В разрезе данной аналитики для нас этот факт не важен. Однако, в случае изменения критериев, возможно, их нужно будет исключить из анализа.

##### Время прослушивания <a id='time'></a>

In [41]:
df['time'].sort_values().unique()

array(['08:00:00', '08:00:02', '08:00:03', ..., '22:00:57', '22:00:58',
       '22:00:59'], dtype=object)

У нас имеется информация о прослушивании музыки пользователями с 08:00:00 до 22:00:59.

##### День недели <a id='day'></a>

In [42]:
df['weekday'].unique()

array(['Wednesday', 'Friday', 'Monday'], dtype=object)

Данные о прослушивании музыки представлены за Понедельник, Среду и Пятницу.

После преобразования всех данных ещё раз проверим отсутствие дубликатов и пропусков. Убеждаемся, что чистка выполнена успешно.

In [43]:
df.duplicated().sum()

0

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      60126 non-null  object
 1   track_name   60126 non-null  object
 2   artist_name  60126 non-null  object
 3   genre_name   60126 non-null  object
 4   city         60126 non-null  object
 5   time         60126 non-null  object
 6   weekday      60126 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


### Вывод <a id='step_1.4'></a>

У нас имеются данные с информацией о прослушивании музыки пользователями. При этом каждое из зафиксированных в данных прослушиваний характеризуется семью признаками: идентификатором пользователя, названием трека, именем исполнителя трека, названием жанра трека, городом, временем и днём недели, в которые пользователь слушал трек.

На этапе предобработки в данных обнаружились пропуски и проблемы с названиями столбцов, а также "явные" и "неявные" дубликаты, которые были обработаны.

После этапа предобработки имеем слеующие данные для анализа. В данных встречается 60 126 прослушиваний музыки, в которых 41 315 уникальных пользователей. Из этих пользователей 29 043 человека прослушивали музыку в Москве, а 12 305 - в Санкт-Петербурге. Поскольку число уникальных пользователей в выборке меньше суммарного значения пользователей из Москвы и Санкт-Петербурга выше, это говорит о том, что некоторые пользователи слушали музыку и в Москве, и в Санкт-Петербурге. В разрезе данной аналитики для нас этот факт не важен. Однако, в случае изменения критериев, возможно, их нужно будет исключить из анализа.

В данных встречается 46 910 уникальных композиций по названию трека и 43 344 уникальных исполнителя. Отметим, что 104 пользователя слушали более 10 треков с разными названиями, а 98 пользователей слушали более 10 треков с разными исполнителями. При этом некоторые из них слушали до 42 треков с разными названиями и до 38 треков с разными исполнителями. Также у нас имеется около 31 800 пользователей, которые включали либо только один трек, либо треки, имеющие одинаковые названия, либо треки, принадлежащие одному исполнителю. Кроме того, данные содержат композиции, принадлежащие к 280 разным жанрам. 

Вся информация представлена за Понедельники, Среды и Пятницы и временной период с 08:00:00 до 22:00:59.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице

Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

Для этого применим последовательную фильтрацию с логической индексацией.

In [63]:
def number_tracks(day, city):
    track_list = df[(df['weekday'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [64]:
number_tracks('Monday', 'Moscow')# количество прослушиваний в Москве по понедельникам

15347

In [65]:
number_tracks('Monday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по понедельникам

5519

In [66]:
number_tracks('Wednesday', 'Moscow')# количество прослушиваний в Москве по средам

10865

In [67]:
number_tracks('Wednesday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по средам

6913

In [68]:
number_tracks('Friday', 'Moscow')# количество прослушиваний в Москве по пятницам

15680

In [69]:
number_tracks('Friday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по пятницам

5802

In [74]:
column = ['city', 'monday', 'wednesday', 'friday'] # Таблица с результатами
data = [
    ['Moscow', '15347', '10865', '15680'],
    ['Saint-Petersburg', '5519', '6913', '5802'],
]
info = pd.DataFrame(data=data, columns=column)

print(info)

               city monday wednesday friday
0            Moscow  15347     10865  15680
1  Saint-Petersburg   5519      6913   5802


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [71]:
# получим таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [72]:
# получим таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* датафрейм с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция вернет информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [73]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в
# заданное время:
# 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
#    которых одновременно:
#    - значение в столбце day равно значению аргумента day
#    - значение в столбце time больше значения аргумента time1
#    - значение в столбце time меньше значения аргумента time2
#    Используйте последовательную фильтрацию с помощью логической индексации.
# 2) сгруппировать датафрейм genre_df по столбцу genre, взять один из его
#    столбцов и посчитать методом count() количество записей для каждого из
#    присутствующих жанров, получившийся Series записать в переменную
#    genre_df_count
# 3) отсортировать genre_df_count по убыванию встречаемости и сохранить
#    в переменную genre_df_sorted
# 4) вернуть Series из 10 первых значений genre_df_sorted, это будут топ-10
#    популярных жанров (в указанный день, в заданное время)

def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['weekday'] == day] # ваш код здесь
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] > time1] # ваш код здесь
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time'] < time2] # ваш код здесь
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre_name')['genre_name'].count() # ваш код здесь
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False) # ваш код здесь
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [75]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         285
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre_name, dtype: int64

In [76]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre_name, dtype: int64

In [77]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre_name, dtype: int64

In [78]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre_name
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre_name, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра

In [80]:
moscow_genres = moscow_general.groupby('genre_name')['genre_name'].count().sort_values(ascending=False)

In [81]:
moscow_genres.head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2094
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

In [82]:
#Аналогично посмотрим по Петербургу
spb_genres = spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending=False)

In [83]:
spb_genres.head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          539
world           515
Name: genre_name, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 

## Итоги исследования

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных.